# Convolutional Neural Networks (CNNs)

**Note:** 이 `notebook`을 실행하기 이전에 다음의 내용을 설명하고 진행하겠습니다. 
 1. Convolution layer
 2. Filter
 3. Stride
 4. Padding
 5. Max-pooling
 6. mnist 데이터

**Note:** 아래 스크립트는 [이 곳](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py)의 내용을 참고하였습니다. 

아래의 스크립트는 `mnist` 데이터를 **CNN** 모델로 분류하는 스크립트입니다. 12 epoch 이후에 99.25%의 성능을 보였습니다. 

## MNIST 데이터

MNIST 데이터는 손으로 쓰여진 숫자들의 이미지로 구성되어 있습니다. 데이터를 분류하는 목적은, 손으로 쓰여진 우편번호를 컴퓨터로 인식하기 위함이었습니다. 

## Import modules

In [2]:
from __future__ import print_function
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Set parameters

In [3]:
np.random.seed(1337)  # for reproducibility

batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

## Import mnist data, and create training/test data

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

11493376/11490434 [==============================] - 58s 5us/step
X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## One-hot encoding

In [5]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

for i in range(0, 10):
    print(y_train[i], "\t", Y_train[i])

5 	 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
0 	 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
4 	 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
1 	 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
9 	 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
2 	 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
1 	 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
3 	 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
1 	 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
4 	 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


## Build a model

In [6]:
model = Sequential()

model.add(Conv2D(nb_filters, kernel_size, padding='valid', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(nb_filters, kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))


## Flatten the output of convolution layer, and send the output to fully-connected layer. 

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
metrics=['accuracy'])

## Train the model

In [7]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq = 0, 
                         write_graph=True, write_images=False)

model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
verbose=1, validation_data=(X_test, Y_test), callbacks = [tensorboard])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 102s 2ms/step - loss: 0.3007 - acc: 0.9066 - val_loss: 0.0649 - val_acc: 0.9795
Epoch 2/12
60000/60000 [==============================] - 105s 2ms/step - loss: 0.1031 - acc: 0.9694 - val_loss: 0.0435 - val_acc: 0.9848
Epoch 3/12
60000/60000 [==============================] - 105s 2ms/step - loss: 0.0762 - acc: 0.9773 - val_loss: 0.0415 - val_acc: 0.9853
Epoch 4/12
60000/60000 [==============================] - 105s 2ms/step - loss: 0.0653 - acc: 0.9805 - val_loss: 0.0326 - val_acc: 0.9888
Epoch 5/12
60000/60000 [==============================] - 107s 2ms/step - loss: 0.0566 - acc: 0.9834 - val_loss: 0.0326 - val_acc: 0.9881
Epoch 6/12
60000/60000 [==============================] - 107s 2ms/step - loss: 0.0521 - acc: 0.9846 - val_loss: 0.0346 - val_acc: 0.9890
Epoch 7/12
60000/60000 [==============================] - 110s 2ms/step - loss: 0.0483 - acc: 0.9852 - val_loss: 0.0289 - 

## Evaluate the model

In [8]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.02962192776228476
Test accuracy: 0.9909
